In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, date

In [2]:
STOCKS_FILE = 'inter_stocks_clean.csv'
DOLLAR_PRICE_FILE = 'dollar_prices_clean.csv'
TWEETS_FILE = 'inter_tweets_clean.csv'

In [3]:
data_stocks = pd.read_csv(STOCKS_FILE, sep=',')
data_dollar = pd.read_csv(DOLLAR_PRICE_FILE, sep=',')
tweets = pd.read_csv(TWEETS_FILE, sep=',')

In [4]:
merged_data = data_stocks[data_stocks['date'] < "2019-10-19 00:00:00"]
merged_data.drop(merged_data.columns[[0]], axis=1, inplace=True)

c:\programdata\anaconda3\envs\stock-market\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
merged_data['dollar_open'] = np.nan
merged_data['dollar_high'] = np.nan
merged_data['dollar_low'] = np.nan
merged_data['dollar_close'] = np.nan
merged_data.sort_values('date', inplace=True)

for i in merged_data.index:
    current_date = merged_data.at[i, 'date']
    current_day, current_time = current_date.split(' ')
    hour, min_sec = current_time.split(':', 1)

    if current_day == "2019-09-23":
        dollar_minuto_inform = data_dollar.loc[data_dollar['date'] == "2019-09-20 21:00:00"]
    else:
        dollar_minuto_inform = data_dollar.loc[data_dollar['date'] == current_day + ' ' + str(int(hour) + 4) + ':' + min_sec]

    merged_data.at[i, 'dollar_open'] = dollar_minuto_inform.iloc[0]['open']
    merged_data.at[i, 'dollar_high'] = dollar_minuto_inform.iloc[0]['high']
    merged_data.at[i, 'dollar_low'] = dollar_minuto_inform.iloc[0]['low']
    merged_data.at[i, 'dollar_close'] = dollar_minuto_inform.iloc[0]['close']

c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [6]:
merged_data['sentiment_polarity'] = 0.0
merged_data['sentiment_subjectivity'] = 0.0
merged_data.head()

c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,date,open,high,low,close,volume,dollar_open,dollar_high,dollar_low,dollar_close,sentiment_polarity,sentiment_subjectivity
0,1,2019-09-10 09:01:00,18.3500,18.5500,18.3500,18.5300,0,4.1002,4.1009,4.0957,4.0979,0.0,0.0
1,2,2019-09-10 09:02:00,18.5307,18.5874,18.4373,18.4607,15063,4.0957,4.0990,4.0957,4.0990,0.0,0.0
2,3,2019-09-10 09:03:00,18.4500,18.4500,18.3400,18.3500,13800,4.0993,4.0997,4.0962,4.0997,0.0,0.0
3,4,2019-09-10 09:04:00,18.3303,18.3759,18.2758,18.3203,47357,4.0982,4.1009,4.0962,4.0977,0.0,0.0
4,5,2019-09-10 09:05:00,18.3506,18.3979,18.3378,18.3606,28101,4.0975,4.1011,4.0975,4.1002,0.0,0.0


In [7]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2019,9,9)
end_dt = date(2019,10,18)

weekdays = [6,7]
valid_dates = []
for dt in daterange(start_dt, end_dt):
    if dt.isoweekday() not in weekdays:
        valid_dates.append(dt.strftime("%Y-%m-%d"))

for i in range(1, len(valid_dates)):
    tweets_of_day = tweets.loc[(tweets['date'] > "{:s} 23:59:59".format(valid_dates[i - 1])) &
                               (tweets['date'] <= "{:s} 23:59:59".format(valid_dates[i]))]
    
    merged_data.loc[(merged_data['date'] >= "{:s} 23:59:59".format(valid_dates[i - 1])) & (merged_data['date'] <= "{:s} 23:59:59".format(valid_dates[i])), 'sentiment_polarity'] = tweets_of_day['sentiment_polarity'].mean()
    merged_data.loc[(merged_data['date'] >= "{:s} 23:59:59".format(valid_dates[i - 1])) & (merged_data['date'] <= "{:s} 23:59:59".format(valid_dates[i])), 'sentiment_subjectivity'] = tweets_of_day['sentiment_subjectivity'].mean()

c:\programdata\anaconda3\envs\stock-market\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
merged_data.to_csv(STOCKS_FILE.split('_', 1)[0] + "_dataset.csv", sep=',', encoding='utf-8')
